In [0]:
import datetime
starttime = datetime.datetime.now()


In [0]:
!export GOOGLE_APPLICATION_CREDENTIALS="My Project-ba13d9d9d219.json"
!pip install google.cloud.speech
# [START speech_quickstart]
import io
import os

# Imports the Google Cloud client library
# [START speech_python_migration_imports]
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
# [END speech_python_migration_imports]

# Instantiates a client
# [START speech_python_migration_client]

client = speech.SpeechClient.from_service_account_json(
      'My Project-ba13d9d9d219.json'
)
# [END speech_python_migration_client]


  Using cached https://files.pythonhosted.org/packages/fa/88/d48588464e8f88b0c8e6999cf32f75ddc43ff4ff1af4e21eb23a61b707fd/google_cloud_speech-1.3.0-py2.py3-none-any.whl


In [0]:
!pip install pydub
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [0]:
problemselect = []
for i in range(1,101):
        if(i<10):
            index = '00'+repr(i)

        if(i>10):
            index = '0'+repr(i)
        if(i==100):
            index = repr(i)
        sound = AudioSegment.from_mp3('C/C0000'+index+'.wav')
        loudness = sound.dBFS
        #print(loudness)

        minsilence=450
        chunks = split_on_silence(sound,
            # must be silent for at least half a second,沉默半秒
            min_silence_len=minsilence,
        
            # consider it silent if quieter than -16 dBFS
            silence_thresh=-45,
            keep_silence=400)
        while(len(chunks)<4):
            minsilence = minsilence-5
            chunks = split_on_silence(sound,
            # must be silent for at least half a second,沉默半秒
            min_silence_len=minsilence,
        
            # consider it silent if quieter than -16 dBFS
            silence_thresh=-45,
            keep_silence=400)

        while(len(chunks)>4):
            minsilence = minsilence+5
            chunks = split_on_silence(sound,
            # must be silent for at least half a second,沉默半秒
            min_silence_len=minsilence,
        
            # consider it silent if quieter than -16 dBFS
            silence_thresh=-45,
            keep_silence=400)
        

        if(len(chunks)!=4):
          problemselect.append(i)

        # # 放弃长度小于2秒的录音片段
        # for i in list(range(len(chunks)))[::-1]:
        #     if len(chunks[i]) <= 0 or len(chunks[i]) >= 10000:
        #         chunks.pop(i)
        # #print('取有效分段(大于2s小于10s)：', len(chunks))
        

        for i, chunk in enumerate(chunks):
            if(len(chunks)==4):
              chunk.export("Csplit/chunk"+index+"of"+repr(i)+".wav" ,format="wav")

   

In [0]:
problemselect

[]

In [0]:

config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='zh-tw',
        audio_channel_count=1,
        enable_automatic_punctuation=False
        )

print(config)

encoding: LINEAR16
sample_rate_hertz: 16000
language_code: "zh-tw"
audio_channel_count: 1



In [0]:
num = 100

In [0]:
import pandas as pd 
articlecsv = pd.read_csv("xunfei.csv") 

In [0]:
from langconv import *

from langconv import *

def Simplified2Traditional(sentence):
    '''
    将sentence中的简体字转为繁体字
    :param sentence: 待转换的句子
    :return: 将句子中简体字转换为繁体字之后的句子
    '''
    sentence = Converter('zh-hant').convert(sentence)
    return sentence

In [0]:
#select
article = []    
question = []
select = [[],[],[],[]]
from pandas.core.frame import DataFrame

for i in range(1,num+1):

  if(i<10):
      index = '00'+repr(i)
  if(i>10):
      index = '0'+repr(i)
  if(i==100):
      index = repr(i)   

  for j in range(0,3):

    if(j!=2):
        if(j==0):
          file = 'A'
          article.append(articlecsv['article'][i-1])

        if(j==1):
          file = 'B'
          file_name = os.path.join(
          os.path.dirname('__file__'),
          file,
          file+'0000'+index+'.wav')
          
          with io.open(file_name, 'rb') as audio_file:
              content = audio_file.read()
              audio = types.RecognitionAudio(content=content)
          response = client.recognize(config, audio)

        
          for result in response.results:
            # if(j==0):
            #   article.append(result.text)
            if(j==1):
              question.append(Simplified2Traditional(result.alternatives[0].transcript))
              #question.append(result.alternatives[0].transcript)

    if(j==2):
      file = 'Csplit'
 
      else:
        for k in range(0,4):
            if(i in problemselect):
              select[k].append('error')
              
            file_name = os.path.join(
              os.path.dirname('__file__'),
              file,
              'chunk'+index+'of'+repr(k)+'.wav')
            #print(file_name)
            with io.open(file_name, 'rb') as audio_file:
              content = audio_file.read()
              audio = types.RecognitionAudio(content=content)
              #print(type(audio))
            response = client.recognize(config, audio)
            for result in response.results:
              select[k].append(Simplified2Traditional(result.alternatives[0].transcript))
              #select[k].append(result.alternatives[0].transcript)
            

      

  if(len(article)==len(question)==len(select[0])):
    c={
      "0" : article,
      "1" : question,
      '2' : select[0],
      '3' : select[1],
      "4" : select[2],
      "5" : select[3],

      }
      #将列表a，b转换成字典
    fsddata=DataFrame(c)#将字典转换成为数据框
    print(len(fsddata))
    fsddata.to_csv(path_or_buf="soundresult.csv",encoding='UTF-8',index=False)


1
2
3
4
5
6
7
8
9
10
11
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [0]:
!pip install https://github.com/huggingface/transformers/releases/download/v0.1.2/pytorch_pretrained_bert-0.1.2.tar.gz

     |████████████████████████████████| 51kB 275kB/s 
  Created wheel for pytorch-pretrained-bert: filename=pytorch_pretrained_bert-0.1.2-cp36-none-any.whl size=32311 sha256=75067be3604a3e63992fbbadd967d76f26e124b213d094e828d0913d786bec51
  Stored in directory: /root/.cache/pip/wheels/95/f2/55/b2c19f6d3a9bbede0fb2d8874b33510e5fe96f33a8549166c9
Successfully built pytorch-pretrained-bert


In [0]:
!nvidia-smi
!rm -rf bert_chinese*
!rm -rf pytorch_pretrained_bert*
!wget -q --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1SXz4Hr073OCuIOqzmi1Ool6m9-g8yYOf' -O bert_chinese.zip
!unzip -q -o bert_chinese.zip
!wget -q --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=15vkVWzKDlik_l2MAkvGI6tIhPauQiGaW' -O pytorch_pretrained_bert.zip
!unzip -q -o pytorch_pretrained_bert.zip
# !wget --no-check-certificate -r 'https://drive.google.com/uc?export=download&id=1IIVjgGX_JeXBNQv-vRAj1JLFvv2x9EVu' -O pytorch_pretrained_bert-0.1.2.zip
# !unzip pytorch_pretrained_bert-0.1.2.zip
# %cd pytorch_pretrained_bert-0.1.2
# !python setup.py install
# %cd ..

Fri Nov 29 15:31:29 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import numpy as np
from tqdm import tqdm


def to_list(tensor):
    return tensor.detach().cpu().tolist()
 

def _get_best_indexes(logits, n_best_size=1):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes
 
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)
    
def predict(eval_dataloader, model, tokenizer, device='cpu'):


    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions = []
    for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc='Iteration'):
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        
        with torch.no_grad():
            print(input_ids.shape)
#             tmp_eval_loss, logits = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask, label_ids)

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        
        logits = softmax(logits.tolist())
        for logit in logits:
            predictions.append(logit)

    return np.argmax(predictions[0])
  

In [0]:
import collections

from torch.utils.data import TensorDataset, DataLoader, SequentialSampler


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

        
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
            
            
def convert_examples_to_features(context, question, choices, tokenizer, ans, max_seq_length=300, label_list=[1,2,3,4]):
    
    text_bs = list()
    for choice in choices:
        # text_b = tokenization.convert_to_unicode(str(row['choice%d' % i]))
        text_bs.append("{} {}".format(question, choice))
        #text_b.append(choice)
      

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i    

    multiple_choice_input_ids = []
    multiple_choice_input_masks = []
    multiple_choice_segment_ids = []
    for text_b in text_bs:  
    

        tokens_a = tokenizer.tokenize(context)  

        tokens_b = None
        if text_b:
            tokens_b = tokenizer.tokenize(text_b)   

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)] 

        
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)   

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)   

        input_ids = tokenizer.convert_tokens_to_ids(tokens) 

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)   

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)   

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        multiple_choice_input_ids.append(input_ids)
        multiple_choice_input_masks.append(input_mask)
        multiple_choice_segment_ids.append(segment_ids)
    assert len(multiple_choice_input_ids) == 4
    assert len(multiple_choice_input_masks) == 4
    assert len(multiple_choice_segment_ids) == 4    

    label_id = label_map[ans] 
    
    

    f = InputFeatures(input_ids=multiple_choice_input_ids,
                      input_mask=multiple_choice_input_masks,
                      segment_ids=multiple_choice_segment_ids,
                      label_id=label_id)  
    
    

    all_input_ids = torch.tensor([f.input_ids], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    eval_dataloader = DataLoader(eval_data, sampler=SequentialSampler(eval_data), batch_size=1)

    return eval_dataloader, tokens




In [0]:
# 因為要我們今天要跑的是中文QA 所以只有Bert可以用
import torch
from pytorch_pretrained_bert import (BertConfig, BertForSequenceClassification,
                                 BertTokenizer)

device = torch.device("cuda")
 
model_name = 'bert_chinese'
config_class, tokenizer_class = BertConfig, BertTokenizer
# model = model_class.from_pretrained('bert_chinese').to(device)
model = torch.load(model_name + '/model.cpt') 
tokenizer = torch.load(model_name + '/tokenizer.cpt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'pytorch_pretrained_bert.modeling.BertForSequenceClassification' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the obj

In [0]:
#import pandas as pd 
#soundresult = pd.read_csv("soundresult.csv") 
#trueanswer = pd.read_csv("trueanswer.csv") 
#soundresult = trueanswer

In [0]:
num=100
import pandas as pd 
final = []
soundresult = pd.read_csv("soundresult.csv") 
column= ['2','3','4','5']
count1 = ['1','2','3','4']
count2 = ['一','二','三','四']
for i in range(0,num):
  context = soundresult['0'][i]
  question = soundresult['1'][i]
  #choices=[soundresult['2'][i],soundresult['3'][i],soundresult['4'][i],soundresult['5'][i]]
  
  choices = []

  for j in range(0,4):
    #choices.append((soundresult[column[j]][i])[1:])
    if((soundresult[column[j]][i][0:1])==count1[j]or(soundresult[column[j]][i])[0:1]==count2[j]): 
      choices.append((soundresult[column[j]][i])[1:])
    else:  
     choices.append((soundresult[column[j]][i]))
  label = 3
  data, tokens = convert_examples_to_features(context, question, choices, tokenizer, 2)
  pred = predict(data, model, tokenizer, device=device)
  final.append(pred+1)


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 4, 300])


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 4, 300])


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 4, 300])


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

torch.Size([1, 4, 300])


Iteration: 100%|██████████| 1/1 [00:00<00:00,  4.31it/s]


In [0]:
len(final)

100

In [0]:
# score = 0
# wrong = [
#          [],[],[],[],[],[]
#          ]
# for i in range(0,100):
  
#   if(final[i]==int(trueanswer['答案'][i])):
#     score=score+1
#   if(final[i]!=int(trueanswer['答案'][i])):  
#     print(final[i])
    #print(soundresult.iloc[i, [0,1,2,3,4,5]])
    #print(trueanswer['tr'][i])
    #print(trueanswer.iloc[i, [1,2,3,4,5,6]])
    # wrong[0].append(soundresult[0][i])
    # wrong[1].append(soundresult[1][i])
    # for j in range(0,4):
    #   wrong[j+2].append(soundresult[column[j]][i])
    #print(soundresult['0'][i]+soundresult['1'][i]+soundresult['2'][i]+soundresult['3'][i]+soundresult['4'][i]+soundresult['5'][i])
    #print(i)
      #wrong.append(soundresult[j][i])
      #wrong.append(trueanswer[j][i])
    
print(score)
# for i in range(0,6):
#   print(wrong[i])
# if(wrong[0]==wrong[1]==wrong[2]==wrong[3]==wrong[4]==wrong[5]):
#   c={
#     "0" : wrong[0],
#     "1" : wrong[1],
#     '2' : wrong[2],
#     '3' : wrong[3],
#     "4" : wrong[4],
#     "5" : wrong[5],

#     }
    
#     #将列表a，b转换成字典
#   from pandas.core.frame import DataFrame
#   fsddata=DataFrame(c)#将字典转换成为数据框
#   print(len(fsddata)+'%')
#   fsddata.to_csv(path_or_buf="wrong.csv",encoding='UTF-8',index=False)

In [0]:
from pandas.core.frame import DataFrame
finaldict = {
    "ID":range(1,101),
    "Answer":final
    }
finaldataframe=DataFrame(finaldict)#将字典转换成为数据框

finaldataframe.to_csv(path_or_buf="final.csv",index=False)

In [0]:
#long running
endtime = datetime.datetime.now()
print (endtime - starttime)

0:13:01.022233
